# Using COVID19API and Alpha Vantage Data to create a market analysis

In [47]:
#libraries used

import requests
import pandas as pd
import datetime

In [48]:
##api link for get use
##https://documenter.getpostman.com/view/10808728/SzS8rjbc?version=latest

##api link for country codename
##https://documenter.getpostman.com/view/10808728/SzS8rjbc?version=latest#7934d316-f751-4914-9909-39f1901caeb8

##function for pulling new covid cases from the API

def newCasesPerDayFunc(country):
#setting variables for requests
    newCasesPerDay = []
    #country = "japan" ##doesn't work for united-states, united-kingdom,
    url = "https://api.covid19api.com/country/"+country+"/status/confirmed"

    fromdate = "2020-01-01T00:00:00Z"
    todate = "2020-07-13T00:00:00Z"
    payload = {}
    headers= {}
    params = {"from":fromdate, "to":todate}

    #response and json form of response
    response = requests.request("GET", url, headers=headers, data = payload, params=params)
    dataParse = response.json()

    def dailyCasesChange(tdayCases, ydayCases):
        dailyChange = tdayCases - ydayCases
        return dailyChange

    #loop to find data we want
    listLength = len(dataParse)
    for x in range(0, listLength):
        
        #if statement: if not day 1 do the covid cases change calculation
        if x > 0:
            dailyCountryProfile = dataParse[x]
            todaysCases = dailyCountryProfile['Cases']

            ydayCountryProfile = dataParse[x-1]
            ydayCases=ydayCountryProfile['Cases']

            newCasesToday = dailyCasesChange(todaysCases, ydayCases)
            newCasesPerDay.append(newCasesToday)

    return(newCasesPerDay)

In [49]:
startDate = '2020-02-01' #variable for start date of covid cases analysis

#list of all countries daily case data
japanNewCases = newCasesPerDayFunc("japan")
brazilNewCases = newCasesPerDayFunc("brazil")
italyNewCases = newCasesPerDayFunc("italy")
#usaNewCases = newCasesPerDayFunc("")
skNewCases = newCasesPerDayFunc("south-korea")

##Generates a list of all dates
datelist = pd.date_range(startDate, periods=len(japanNewCases)).tolist()

In [57]:
newDateList = []

#loop to change the date formats for table
for item in datelist:
    #print(item)
    #print(type(item))
    newDateFormat = item.strftime("%Y-%m-%d")
    newDateItem = datetime.datetime.strptime(newDateFormat, '%Y-%m-%d').strftime('%Y-%m-%d')
    newDateList.append(newDateItem)
print(newDateList)

['2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29', '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13', '2020-03-14', '2020-03-15', '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-21', '2020-03-22', '2020-03-23', '2020-03-24', '2020-03-25', '2020-03-26', '2020-03-27', '2020-03-28', '2020-03-29', '2020-03-30', '2020-03-31', '2020-04-01', '2020-04-02', '2020-04-03', '2020-04-04', '2020-04-05', '2020-04-06', '2020-04-07', '2020-04-08', '2020-04-09', '2020-04-10', '2020-04-11', '2020

In [58]:
#Create DF based on a dict of lists
#
#
covidCasesDict = {'japanCases':japanNewCases,'brazilCases': brazilNewCases,'italyCases':italyNewCases,'skCases':skNewCases}

casesDF = pd.DataFrame(covidCasesDict, index = pd.to_datetime(newDateList))
print(casesDF)
#casesDF.head()

            japanCases  brazilCases  italyCases  skCases
2020-02-01           0            0           0        0
2020-02-02           0            0           0        1
2020-02-03           0            0           0        0
2020-02-04           2            0           0        1
2020-02-05           0            0           0        1
2020-02-06           3            0           0        0
2020-02-07           0            0           0        0
2020-02-08           4            0           0        0
2020-02-09           4            0           2        7
2020-02-10           5            0           0        1
2020-02-11           0            0           0        3
2020-02-12           0            0           0        0
2020-02-13           2            0           0        1
2020-02-14           1            0           0        3
2020-02-15           0            0           0        4
2020-02-16           0            0           1        1
2020-02-17           1         

In [59]:
casesDF.tail()

,japanCases,brazilCases,italyCases,skCases
2020-07-18,356,42619,214,45
2020-07-19,427,45048,276,35
2020-07-20,386,39023,188,44
2020-07-21,411,24831,234,62
2020-07-22,284,20286,169,33


In [9]:
#Used for pulling the data
import urllib.request 
import json
#Used for formatting the data
import pandas as pd
#Used for data visualisation
import matplotlib.pyplot as plt
from matplotlib import style

In [10]:
#Returns a pandaDF of the closing stock prices in the last x amount of days
def getPriceOfStock(companySymbol, numOfDays): 
    url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol="+companySymbol+"&outputsize=full&apikey=Y11HUWNU7HM58RRL"
    json_obj = urllib.request.urlopen(url)
    #Finds the data from the url
    data = json.load(json_obj)
    #Choosing the revelant key from the data dictionary
    dates = data["Time Series (Daily)"] 
    #Initializing lists to store the data
    final_prices = [] 
    calendar = []
    counter = 0
    #For-loop used to run through every trading day 
    for day in dates: 
        counter += 1
        #Limits the amount of data using the number of days wanted
        if counter > numOfDays: 
            break
        #Gets the stock's price - This will include Open, High, Low, Closed prices
        prices = dates[day]
        #Choosing the stock's closed price
        priceNum = float(prices['4. close']) 
        final_prices.append(priceNum)
        calendar.append(day)
    #Dictionary containing the company and it's stock prices 
    final_prices.reverse()
    calendar.reverse()
    stockDict = {companySymbol : final_prices} #CHANGE
    #Formatting the pulled data to a Panda DataFrame
    df = pd.DataFrame(data = stockDict, index = pd.to_datetime(calendar))
    return df

In [11]:
#NOTE: Alpha Vantage only allows 5 API calls a minute, 500 a day -> Can only have an input size of list size 5
#Function to format (multiple) stocks into a PandaDF and alligns the dates
def bundleStockPrices(arrayOfCompanySymbols, numOfDays):
    largeTable = getPriceOfStock(arrayOfCompanySymbols[0], numOfDays) 
    del arrayOfCompanySymbols[0]
    for symbol in arrayOfCompanySymbols:
        smallTable = getPriceOfStock(symbol, numOfDays)
        #Concatenating the tables together -> will provide the intersection of all the data -> will provide <= 200 day data values
        #largeTable = pd.concat([largeTable, smallTable.reindex(largeTable.index)], axis = 1)
        largeTable = pd.concat([largeTable, smallTable], axis = 1, join ='inner')
    print("Any null data: "+str(largeTable.isnull().values.any()))
    print(largeTable.head())
    print("...")
    print(largeTable.tail())
    return largeTable

In [21]:
country_etf = bundleStockPrices(['JPXN','EWY'],200)
#'EWZ','EWI'

Any null data: False
               JPXN    EWY
2019-09-27  60.7093  55.43
2019-09-30  61.1200  56.34
2019-10-01  60.6445  56.04
2019-10-02  59.9415  55.02
2019-10-03  60.2398  55.74
...
             JPXN    EWY
2020-07-08  59.69  59.18
2020-07-09  59.35  58.82
2020-07-10  60.08  58.43
2020-07-13  59.84  58.52
2020-07-14  60.34  59.05


In [62]:
print(casesDF)
print(country_etf)

covidAndStocks = pd.concat([country_etf, casesDF], axis=1, join='inner')
#print(covidAndStocks)

            japanCases  brazilCases  italyCases  skCases
2020-02-01           0            0           0        0
2020-02-02           0            0           0        1
2020-02-03           0            0           0        0
2020-02-04           2            0           0        1
2020-02-05           0            0           0        1
2020-02-06           3            0           0        0
2020-02-07           0            0           0        0
2020-02-08           4            0           0        0
2020-02-09           4            0           2        7
2020-02-10           5            0           0        1
2020-02-11           0            0           0        3
2020-02-12           0            0           0        0
2020-02-13           2            0           0        1
2020-02-14           1            0           0        3
2020-02-15           0            0           0        4
2020-02-16           0            0           1        1
2020-02-17           1         